In [1]:
from pynse import *
import os
import datetime
import matplotlib.pyplot as plt
#import mplfinance as mpf
import time, asyncio
import matplotlib.dates as mdates
import mplfinance as mpf
import statsmodels.api as sm


import os
import glob
import pandas as pd
import mplfinance as mpf

nse=Nse()
#nse.bhavcopy(series="all")
#bhavcopy_full=nse.bhavcopy(series="all")

Loading DATA

In [2]:
# Quarterly Result

folder_path = "C:/Users/99111/OneDrive/Desktop/Full data set/Financial Reports/Quarterly Results cons from 200001 - 202212/*.csv"

files = glob.glob(folder_path)

df_list = []

for file in files:
    df = pd.read_csv(file)
    df_list.append(df)

results = pd.concat(df_list)

results['QTRC_Date End'] = pd.to_datetime(results['QTRC_Date End'], format='%Y%m')
results['QTRC_Date End'] = results['QTRC_Date End'] + pd.offsets.MonthEnd(1)
results['QTRC_Date End'] = results['QTRC_Date End'].dt.date

results['QTRC_Result Date'] = pd.to_datetime(results['QTRC_Result Date'], format='%d-%b-%Y')
results['QTRC_Result Date'] = results['QTRC_Result Date'].dt.date

results = results.sort_values('QTRC_Date End')
results.fillna(0, inplace=True)  # Fill NaN values with zeroes
results.replace('NaT', np.datetime64('NaT'), inplace=True)
symbols = results['Company Name'].unique().tolist()

results['uniquecode'] = results['Company Name'] + results['QTRC_Date End'].astype(str)
results = results.drop_duplicates(subset=['uniquecode'])

results = results[results['QTRC_Date End'] >= pd.to_datetime('2016-01-01')].copy()
results

C:\Users\99111\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pandas\core\ops\array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)


,Sr.No,Accord Code,Company Name,CD_Trading Status,CD_ISIN No,CD_BSE Code,CD_NSE Symbol,QTRC_Date End,QTRC_Gross Sales,QTRC_Less: Excise Duty,...,QTRC_PBTM%,QTRC_PATM%,QTRC_Result Date,QTRC_Date range,QTRC_Year,QTRC_Dividend (%),QTRC_Other Income.1,Unnamed: 87,Unnamed: 88,uniquecode
14943,14944,219897,Talwalkars Better Value Fitness Ltd.,InActive,INE502K01016,533200.0,TALWALKARS,2016-03-31,78.24,0.0,...,45.52,21.60,2016-05-05,29-Jun-2016,2016.0,0.0,0.00,0.0,0.0,Talwalkars Better Value Fitness Ltd.2016-03-31
16809,16810,132721,Visa Steel Ltd.,Active,INE286H01012,532721.0,VISASTEEL,2016-03-31,336.03,0.0,...,-46.85,-54.97,2016-05-27,29-Jun-2016,2016.0,0.0,1.30,0.0,0.0,Visa Steel Ltd.2016-03-31
7222,7223,132894,Indowind Energy Ltd.,Active,INE227G01018,532894.0,INDOWIND,2016-03-31,2.22,0.0,...,-12.35,12.22,2016-05-26,29-Jun-2016,2016.0,0.0,4.38,0.0,0.0,Indowind Energy Ltd.2016-03-31
123,124,111339,Abhayam Trading Ltd.,Suspended,INE212A01013,511339.0,APPLECREDT,2016-03-31,0.00,0.0,...,0.00,0.00,2017-04-24,29-Jun-2016,2016.0,0.0,0.00,0.0,0.0,Abhayam Trading Ltd.2016-03-31
11688,11689,123539,Precision Wires India Ltd.,Active,INE372C01037,523539.0,PRECWIRE,2016-03-31,209.58,0.0,...,3.55,2.36,2016-05-26,29-Jun-2016,2016.0,0.0,0.14,0.0,0.0,Precision Wires India Ltd.2016-03-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1038,1039,215367,Jubilant FoodWorks Ltd.,Active,INE797F01020,533155.0,JUBLFOOD,2023-12-31,1378.12,0.0,...,5.10,3.64,2024-01-31,0,2023.0,0.0,4.16,0.0,0.0,Jubilant FoodWorks Ltd.2023-12-31
1040,1041,301578,Jubilant Ingrevia Ltd.,Active,INE0BY001018,543271.0,JUBLINGREA,2023-12-31,960.45,0.0,...,5.70,3.99,2024-01-30,0,2023.0,0.0,8.78,0.0,0.0,Jubilant Ingrevia Ltd.2023-12-31
1041,1042,130019,Jubilant Pharmova Ltd.,Active,INE700A01033,530019.0,JUBLPHARMA,2023-12-31,1665.20,0.0,...,5.24,3.18,2024-02-02,0,2023.0,0.0,35.80,0.0,0.0,Jubilant Pharmova Ltd.2023-12-31
1032,1033,132642,JSW Holdings Ltd.,Active,INE824G01012,532642.0,JSWHL,2023-12-31,27.29,0.0,...,90.08,66.75,2024-01-24,0,2023.0,0.0,0.00,0.0,0.0,JSW Holdings Ltd.2023-12-31


Companies Quarterly result

In [3]:

excel = pd.DataFrame()

for symbol in symbols:
    try:
        # Execute the code for each Company
        results1 = results[results['Company Name'] == symbol].copy()
        results1 = results1.sort_values('QTRC_Date End')

        #Expenditure and sales flag    
        results1['Increaseinstock'] = -1 * (results1['QTRC_(Increase) / Decrease In Stocks'][results1['QTRC_(Increase) / Decrease In Stocks'] < 0])
        results1['Increase stock/Netsales'] = results1['Increaseinstock'] / results1['QTRC_Net Sales & Other Operating Income']
        results1['foreignexchangeloans/tExpenditure'] = results1['QTRC_Loss on Foreign Exchange Loan'] / results1['QTRC_Total Expenditure']
        results1['Miscexpense/tExpenditure'] = results1['QTRC_Miscellaneous Expenses'] / results1['QTRC_Total Expenditure']  
        
        #other Income 
        results1['otherincomeFlag'] = results1['QTRC_Other Income'] / results1['QTRC_PBIDT (Excl OI)']  
        results1['OffloadingInvst/operatingIncome'] = results1['QTRC_Profit on Sale of Investments'] / results1['QTRC_PBIDT (Excl OI)']  
        results1['OffloadingAssets/operatingIncome'] = results1['QTRC_Profit on Sale of Assets'] / results1['QTRC_PBIDT (Excl OI)']
        results1['provisions/OperatingIncome'] = results1['QTRC_Provision Written Back'] / results1['QTRC_PBIDT (Excl OI)']  
        results1['otherincomeFlag'] = results1['QTRC_Provision Written Back'] / results1['QTRC_PBIDT (Excl OI)']  

        #Profit Deductions
        results1['ShadyDeductionsonOperatingIncome'] = (results1['QTRC_Exceptional Items'] + results1['QTRC_Foreign Exchange Gain / Loss'] + results1['QTRC_Other Exceoptional Items']) / results1['QTRC_Operating Profit']  
        results1['ShadyDeductionsonPBT'] = results1['QTRC_Provisions and Contingencies']  / results1['QTRC_PBT'] 
        results1['ShadyDeductionsonPAT'] = (results1['QTRC_Extraordinary Items'] + results1['QTRC_Prior Period Expenses'] + results1['QTRC_Other Adjustments'] + results1['QTRC_Other related items'] + results1['QTRC_Misc. Expenses Written off']) / results1['QTRC_Profit after tax'] 
        results1['MinorityInterestFlag'] = results1['QTRC_Minority Interest']  / results1['QTRC_Profit after tax'] 

        results1['SalesTTM'] = results1['QTRC_Net Sales & Other Operating Income'].rolling(window=4).sum()
        results1['OperatingprofitTTM'] = results1['QTRC_Operating Profit'].rolling(window=4).sum()
        results1['NetProfit_TTM'] = results1['QTRC_Consolidated Net Profit'].rolling(window=4).sum()

        results1['SalesTTM_change'] = (results1['SalesTTM'] - results1['SalesTTM'].shift(1)) / results1['SalesTTM'].shift(1)
        results1['OperatingprofitTTM_change'] = (results1['OperatingprofitTTM'] - results1['OperatingprofitTTM'].shift(1)) / results1['OperatingprofitTTM'].shift(1)
        results1['NetProfit_TTM_change'] = (results1['NetProfit_TTM'] - results1['NetProfit_TTM'].shift(1)) / results1['NetProfit_TTM'].shift(1)

        results1['SalesTTM_Absolute'] = results1['SalesTTM'] - results1['SalesTTM'].shift(1)
        results1['OperatingprofitTTM_Absolute'] = results1['OperatingprofitTTM'] - results1['OperatingprofitTTM'].shift(1)
        results1['NetProfit_TTM_change_Absolute'] = results1['NetProfit_TTM'] - results1['NetProfit_TTM'].shift(1)

        results1['NetProfit+Flagger'] = np.where(results1['NetProfit_TTM_change'] > 0, results1['NetProfit_TTM_change'], 0)
        results1['NetProfit-lagger'] = np.where(results1['NetProfit_TTM_change'] < 0, results1['NetProfit_TTM_change'], 0)

        results1['SalesGrowth_Q'] = (results1['QTRC_Net Sales & Other Operating Income'] - results1['QTRC_Net Sales & Other Operating Income'].shift(1)) / results1['QTRC_Net Sales & Other Operating Income'].shift(1)
        results1['OperatingGrowth'] = (results1['QTRC_Operating Profit'] - results1['QTRC_Operating Profit'].shift(1)) / results1['QTRC_Operating Profit'].shift(1)
        results1['OperatingGrowth(Excl OI)'] = (results1['QTRC_PBIDT (Excl OI)'] - results1['QTRC_PBIDT (Excl OI)'].shift(1)) / results1['QTRC_PBIDT (Excl OI)'].shift(1)
        results1['NetprofitGrowth_Q'] = (results1['QTRC_Consolidated Net Profit'] - results1['QTRC_Consolidated Net Profit'].shift(1)) / results1['QTRC_Consolidated Net Profit'].shift(1)

        results1['symbol'] = symbol
        excel = pd.concat([excel, results1])

    except Exception as e:
        print(f"Error occurred  for {symbol}: {str(e)}")
        continue    
        
# Step 4: Export the collated dataframe to an XLS file
excel['NetProfit_TTM_Positives'] = excel['NetProfit_TTM'].apply(lambda x: x if x > 0 else 0)
excel['NetProfit_Quarterly_Positives'] = excel['QTRC_Consolidated Net Profit'].apply(lambda x: x if x > 0 else 0)   
excel['uniquecode'] = excel['Company Name'] + excel['QTRC_Date End'].astype(str)

filepath = r'C:\Users\99111\OneDrive\Desktop\Full data set\Quarterly results processed\Quarterlyresults.xlsx'
excel.to_excel(filepath, index=False)

#QuarterlyStocks = pd.DataFrame()
#TTMStocks = pd.DataFrame()

#dates = excel['QTRC_Date End'].unique().tolist()

#for date in dates:
#    try:
#        excel1 = excel[(excel['QTRC_Date End'] == date)]
#
#        excelQRT = excel1.sort_values('NetProfit_Quarterly_Positives', ascending=False)
#        QuarterlyProfit = excelQRT['NetProfit_Quarterly_Positives'].sum()*.99
#        excelQRT['cumulative_Profit'] = excelQRT['NetProfit_Quarterly_Positives'].cumsum()
#        excelQRT['absdiff'] = abs(excelQRT['cumulative_Profit']  - QuarterlyProfit)
#        excelQRT = excelQRT.reset_index(drop=False)                                        
#        ATM_SP_IDX1 = excelQRT['absdiff'].idxmin()
#        excelQRT1 = excelQRT[:ATM_SP_IDX1+1].copy()
#        QuarterlyStocks = pd.concat([QuarterlyStocks, excelQRT1])
#
#        excelTTM = excel1.sort_values('NetProfit_TTM_Positives', ascending=False)
#        TTMProfit = excelTTM['NetProfit_TTM_Positives'].sum()*.99
#        excelTTM['cumulative_Profit'] = excelTTM['NetProfit_TTM_Positives'].cumsum()
#        excelTTM['absdiff'] = abs(excelTTM['cumulative_Profit']  - TTMProfit)
#        excelTTM = excelTTM.reset_index(drop=False)                                        
#        ATM_SP_IDX2 = excelTTM['absdiff'].idxmin()
#        excelTTM1 = excelTTM[:ATM_SP_IDX2+1].copy()
#        TTMStocks = pd.concat([TTMStocks, excelTTM1])

#    except Exception as e:
#        print(f"Error occurred  for {date}: {str(e)}")
#        continue  

#filepath = r'C:\Users\99111\OneDrive\Desktop\Full data set\Quarterly results processed\QuarterlyFilter.xlsx'
#QuarterlyStocks.to_excel(filepath, index=False)

#filepath = r'C:\Users\99111\OneDrive\Desktop\Full data set\Quarterly results processed\TTMFilter.xlsx'
#TTMStocks.to_excel(filepath, index=False)


#SORTING Quarterly results DF's and Fetching MCAP, PE and EV / EBITDA
########################################
#FETCHING QUARTERLY RESULTS

file_path1 = r"C:\Users\99111\OneDrive\Desktop\Full data set\Quarterly results processed\Quarterlyresults.xlsx"

rdf = pd.read_excel(file_path1)
symbols = rdf['Company Name'].unique().tolist()
rdf = rdf[['CD_NSE Symbol', 'QTRC_Date End', 'QTRC_Result Date','QTRC_Reserves','SalesTTM','OperatingprofitTTM','NetProfit_TTM','SalesTTM_change','OperatingprofitTTM_change','NetProfit_TTM_change','SalesTTM_Absolute','OperatingprofitTTM_Absolute','NetProfit_TTM_change_Absolute', 'NetProfit_Quarterly_Positives', 'SalesGrowth_Q','OperatingGrowth','OperatingGrowth(Excl OI)', 'NetprofitGrowth_Q', 'Increaseinstock', 'Increase stock/Netsales', 'foreignexchangeloans/tExpenditure', 'Miscexpense/tExpenditure', 'otherincomeFlag', 'OffloadingInvst/operatingIncome', 'OffloadingAssets/operatingIncome', 'provisions/OperatingIncome', 'ShadyDeductionsonOperatingIncome', 'ShadyDeductionsonPBT', 'ShadyDeductionsonPAT', 'MinorityInterestFlag','uniquecode', 'Company Name', 'QTRC_Net Sales & Other Operating Income', 'QTRC_PBIDT (Excl OI)', 'QTRC_Operating Profit', 'QTRC_Consolidated Net Profit']].copy()

#########################################
#FETCHING LATEST MCAP AND INDUSTRY AND SECTORS

folder_path = "C:/Users/99111/OneDrive/Desktop/Full data set/Price, PE, EBITDA , EV & Mcap/*.csv"

files = glob.glob(folder_path)

df_list = []

for file in files:
    df = pd.read_csv(file)
    df_list.append(df)

Mcap = pd.concat(df_list)

Mcap['NDP_Date'] = pd.to_datetime(Mcap['NDP_Date'])

#Mcap = Mcap.sort_values(by='NDP_Date', ascending=False)
Mcap['uniquecode'] = Mcap['Company Name'] + Mcap['NDP_Date'].astype(str)

Mcap = Mcap.sort_values(by='NDP_Date', ascending=False)
Mcap = Mcap.drop_duplicates(subset='Company Name' , keep= 'first')
Mcap = Mcap[['Company Name', 'NDP_Date','CD_Sector','CD_Industry','NDP_Mcap','NDP_Consolidated PE','NDP_Consolidated EV EBIDTA']].copy()

#########################################

ff = pd.merge(rdf, Mcap, on='Company Name', how='left')

ff['QTRC_Result Date'] = pd.to_datetime(ff['QTRC_Result Date'], errors='coerce')
ff = ff.dropna(subset=['QTRC_Result Date']).copy()
sorted_df = ff.sort_values(by=['QTRC_Result Date', 'NetProfit_Quarterly_Positives'], ascending=[False, False])
sorted_df = sorted_df[['Company Name','CD_NSE Symbol', 'CD_Sector', 'NDP_Mcap', 'NDP_Consolidated PE', 'NDP_Consolidated EV EBIDTA', 'QTRC_Date End', 'QTRC_Result Date','QTRC_Reserves','SalesTTM','OperatingprofitTTM','NetProfit_TTM','SalesTTM_change','OperatingprofitTTM_change','NetProfit_TTM_change','SalesTTM_Absolute','OperatingprofitTTM_Absolute','NetProfit_TTM_change_Absolute', 'NetProfit_Quarterly_Positives', 'SalesGrowth_Q', 'OperatingGrowth','OperatingGrowth(Excl OI)', 'NetprofitGrowth_Q', 'NDP_Consolidated PE', 'NDP_Consolidated EV EBIDTA','Increaseinstock', 'Increase stock/Netsales', 'foreignexchangeloans/tExpenditure', 'Miscexpense/tExpenditure', 'otherincomeFlag', 'OffloadingInvst/operatingIncome', 'OffloadingAssets/operatingIncome', 'provisions/OperatingIncome', 'ShadyDeductionsonOperatingIncome', 'ShadyDeductionsonPBT', 'ShadyDeductionsonPAT', 'MinorityInterestFlag','uniquecode', 'QTRC_Net Sales & Other Operating Income','QTRC_PBIDT (Excl OI)', 'QTRC_Operating Profit', 'QTRC_Consolidated Net Profit']].copy()
sorted_df['uniquecode_1'] = sorted_df['CD_Sector'] + sorted_df['QTRC_Date End'].astype(str)


filepath = r'C:\Users\99111\OneDrive\Desktop\daily updates\Quarterly Results\SortedQuarterlyresults.xlsx'
sorted_df.to_excel(filepath, index=False)

C:\Users\99111\AppData\Local\Temp\ipykernel_8468\3680331057.py:10: FutureWarning: reindexing with a non-unique Index is deprecated and will raise in a future version.
  results1['Increaseinstock'] = -1 * (results1['QTRC_(Increase) / Decrease In Stocks'][results1['QTRC_(Increase) / Decrease In Stocks'] < 0])


Error occurred  for Cyient Ltd.: cannot reindex on an axis with duplicate labels


C:\Users\99111\AppData\Local\Temp\ipykernel_8468\3680331057.py:10: FutureWarning: reindexing with a non-unique Index is deprecated and will raise in a future version.
  results1['Increaseinstock'] = -1 * (results1['QTRC_(Increase) / Decrease In Stocks'][results1['QTRC_(Increase) / Decrease In Stocks'] < 0])


Error occurred  for Dr. Reddy's Laboratories Ltd.: cannot reindex on an axis with duplicate labels


C:\Users\99111\AppData\Local\Temp\ipykernel_8468\3680331057.py:10: FutureWarning: reindexing with a non-unique Index is deprecated and will raise in a future version.
  results1['Increaseinstock'] = -1 * (results1['QTRC_(Increase) / Decrease In Stocks'][results1['QTRC_(Increase) / Decrease In Stocks'] < 0])
C:\Users\99111\AppData\Local\Temp\ipykernel_8468\3680331057.py:10: FutureWarning: reindexing with a non-unique Index is deprecated and will raise in a future version.
  results1['Increaseinstock'] = -1 * (results1['QTRC_(Increase) / Decrease In Stocks'][results1['QTRC_(Increase) / Decrease In Stocks'] < 0])


...

Error occurred  for Dhatre Udyog Ltd.: cannot reindex on an axis with duplicate labels
Error occurred  for G-Tec Jainx Education Ltd.: cannot reindex on an axis with duplicate labels


C:\Users\99111\AppData\Local\Temp\ipykernel_8468\3680331057.py:10: FutureWarning: reindexing with a non-unique Index is deprecated and will raise in a future version.
  results1['Increaseinstock'] = -1 * (results1['QTRC_(Increase) / Decrease In Stocks'][results1['QTRC_(Increase) / Decrease In Stocks'] < 0])


Error occurred  for Lloyds Enterprises Ltd.: cannot reindex on an axis with duplicate labels


C:\Users\99111\AppData\Local\Temp\ipykernel_8468\3680331057.py:10: FutureWarning: reindexing with a non-unique Index is deprecated and will raise in a future version.
  results1['Increaseinstock'] = -1 * (results1['QTRC_(Increase) / Decrease In Stocks'][results1['QTRC_(Increase) / Decrease In Stocks'] < 0])


Error occurred  for Abans Holdings Ltd.: cannot reindex on an axis with duplicate labels


Testing industry growth sheet

In [4]:
#########################################
#FETCHING PROCESSED QUARTERLY RESULTS

file_path1 = r"C:\Users\99111\OneDrive\Desktop\Full data set\Quarterly results processed\Quarterlyresults.xlsx"

rdf = pd.read_excel(file_path1)
symbols = rdf['Company Name'].unique().tolist()
rdf = rdf[['CD_NSE Symbol', 'QTRC_Date End', 'QTRC_Result Date','QTRC_Reserves','SalesTTM','OperatingprofitTTM','NetProfit_TTM','SalesTTM_change','OperatingprofitTTM_change','NetProfit_TTM_change','SalesTTM_Absolute','OperatingprofitTTM_Absolute','NetProfit_TTM_change_Absolute', 'NetProfit_Quarterly_Positives','OperatingGrowth','OperatingGrowth(Excl OI)', 'Increaseinstock', 'Increase stock/Netsales', 'foreignexchangeloans/tExpenditure', 'Miscexpense/tExpenditure', 'otherincomeFlag', 'OffloadingInvst/operatingIncome', 'OffloadingAssets/operatingIncome', 'provisions/OperatingIncome', 'ShadyDeductionsonOperatingIncome', 'ShadyDeductionsonPBT', 'ShadyDeductionsonPAT', 'MinorityInterestFlag','uniquecode','Company Name', 'QTRC_Net Sales & Other Operating Income','QTRC_PBIDT (Excl OI)', 'QTRC_Operating Profit', 'QTRC_Consolidated Net Profit']].copy()

#########################################
#FETCHING LATEST MCAP AND INDUSTRY AND SECTORS

folder_path = "C:/Users/99111/OneDrive/Desktop/Full data set/Price, PE, EBITDA , EV & Mcap/*.csv"

files = glob.glob(folder_path)

df_list = []

for file in files:
    df = pd.read_csv(file)
    df_list.append(df)

Mcap = pd.concat(df_list)

Mcap['NDP_Date'] = pd.to_datetime(Mcap['NDP_Date'])

#Mcap = Mcap.sort_values(by='NDP_Date', ascending=False)
Mcap['uniquecode'] = Mcap['Company Name'] + Mcap['NDP_Date'].astype(str)

Mcap = Mcap.sort_values(by='NDP_Date', ascending=False)
Mcap = Mcap.drop_duplicates(subset='Company Name' , keep= 'first')
Mcap = Mcap[['Company Name', 'NDP_Date','CD_Sector','CD_Industry','NDP_Mcap','NDP_Consolidated PE','NDP_Consolidated EV EBIDTA']].copy()

#########################################
#MERGING THE 2 DATA FRAMES

ff = pd.merge(rdf, Mcap, on='Company Name', how='left')
ff = ff.sort_values(by='QTRC_Date End', ascending=True)
ff['QTRC_Result Date'] = pd.to_datetime(ff['QTRC_Result Date'], errors='coerce')
sectors= ff['CD_Sector'].unique().tolist()
Industries= ff['CD_Industry'].unique().tolist()





##########################################
#SECTORS PROCESSING
##########################################

ff = ff[ff['QTRC_Date End'] >= pd.to_datetime('2016-01-01')]

datesdf = pd.DataFrame()

for sector in sectors:
    try:
        # Filter the main DataFrame for the current sector
        sff = ff[ff['CD_Sector'] == sector].copy()

        # Get unique dates in the sector
        dates = sff['QTRC_Date End'].unique()

        # Create an empty DataFrame for the current sector
        sector_df = pd.DataFrame()

        for date in dates:
            try:
                dff = sff[sff['QTRC_Date End'] == date].copy()
                sym = dff['Company Name'].unique().tolist()

                sff1 = sff[sff['Company Name'].isin(sym)]
                sff1 = sff1[sff1['QTRC_Date End'] <= date].copy()

                sff1_p = sff1[sff1['QTRC_Date End'] == sff1['QTRC_Date End'].sort_values(ascending=False).unique()[1]].copy()

                sff1 = sff1[sff1['QTRC_Date End'] == date].copy()

                df = pd.concat([sff1_p, sff1])

                df['Count'] = df['Company Name'].map(df['Company Name'].value_counts())

                ################################
                #Need to add stuff here
                ################################

                df1 = df[df['Count'] == 1].copy()
                missingOPTTM = df1['OperatingprofitTTM']
                missingMcap = df1['NDP_Mcap']
                missingsymbs = df1.shape[0]

                df2 = df[df['Count'] == 2].copy()
                symcounts = df2.shape[0]
                df2 = df2.groupby('QTRC_Date End')[['SalesTTM', 'OperatingprofitTTM', 'NetProfit_TTM', 'SalesTTM_Absolute', 'OperatingprofitTTM_Absolute', 'NetProfit_TTM_change_Absolute','Increaseinstock', 'NetProfit_Quarterly_Positives', 'QTRC_Net Sales & Other Operating Income','QTRC_PBIDT (Excl OI)', 'QTRC_Operating Profit', 'QTRC_Consolidated Net Profit']].sum()

                df2['salesTTM%'] = ((df2['SalesTTM'] - df2['SalesTTM'].shift(1)) / df2['SalesTTM'].shift(1)) 
                df2['OPTTM%'] = ((df2['OperatingprofitTTM'] - df2['OperatingprofitTTM'].shift(1)) / df2['OperatingprofitTTM'].shift(1)) 
                df2['NetProfitTTM%'] = ((df2['NetProfit_TTM'] - df2['NetProfit_TTM'].shift(1)) / df2['NetProfit_TTM'].shift(1)) 
                
                df2['Sales_Q'] = ((df2['QTRC_Net Sales & Other Operating Income'] - df2['QTRC_Net Sales & Other Operating Income'].shift(1)) / df2['QTRC_Net Sales & Other Operating Income'].shift(1)) 
                df2['OPwithoutOI_Q'] = ((df2['QTRC_PBIDT (Excl OI)'] - df2['QTRC_PBIDT (Excl OI)'].shift(1)) / df2['QTRC_PBIDT (Excl OI)'].shift(1)) 
                df2['OP_Q'] = ((df2['QTRC_Operating Profit'] - df2['QTRC_Operating Profit'].shift(1)) / df2['QTRC_Operating Profit'].shift(1)) 
                df2['Netprofit_Q'] = ((df2['QTRC_Consolidated Net Profit'] - df2['QTRC_Consolidated Net Profit'].shift(1)) / df2['QTRC_Consolidated Net Profit'].shift(1)) 


                df2 = df2.drop(df2.index[0])
                df2['Date'] = date
                df2['sector'] = sector
                
                df2['CompanyCounts'] = symcounts
                df2['MissingSymb'] = missingsymbs
                df2['MissingOPTTM'] = missingOPTTM
                df2['MissingMCAP'] = missingMcap

                sector_df = pd.concat([sector_df, df2])

            except Exception as e:
                print(f"Error occurred for {date} in sector {sector}: {str(e)}")
                continue

        # Concatenate the results for the current sector to the main results DataFrame
        datesdf = pd.concat([datesdf, sector_df])

    except Exception as e:
        print(f"Error occurred for sector {sector}: {str(e)}")
        continue

datesdf['uniquecode_1'] = datesdf['sector'] + datesdf['Date'].astype(str)
filepath = r'C:\Users\99111\OneDrive\Desktop\daily updates\Quarterly Results\IndustryQuarterlyresults.xlsx'
datesdf.to_excel(filepath, index=True)

##########################################
#INDUSTRY PROCESSING
##########################################

datesdf = pd.DataFrame()
1
for sector in Industries:
    try:
        # Filter the main DataFrame for the current sector
        sff = ff[ff['CD_Industry'] == sector].copy()

        # Get unique dates in the sector
        dates = sff['QTRC_Date End'].unique()

        # Create an empty DataFrame for the current sector
        sector_df = pd.DataFrame()

        for date in dates:
            try:
                dff = sff[sff['QTRC_Date End'] == date].copy()
                sym = dff['Company Name'].unique().tolist()

                sff1 = sff[sff['Company Name'].isin(sym)]
                sff1 = sff1[sff1['QTRC_Date End'] <= date].copy()

                sff1_p = sff1[sff1['QTRC_Date End'] == sff1['QTRC_Date End'].sort_values(ascending=False).unique()[1]].copy()

                sff1 = sff1[sff1['QTRC_Date End'] == date].copy()

                df = pd.concat([sff1_p, sff1])

                df['Count'] = df['Company Name'].map(df['Company Name'].value_counts())

                ################################
                #Need to add stuff here
                ################################

                df1 = df[df['Count'] == 1].copy()
                missingOPTTM = df1['OperatingprofitTTM']
                missingMcap = df1['NDP_Mcap']
                missingsymbs = df1.shape[0]

                df2 = df[df['Count'] == 2].copy()
                symcounts = df2.shape[0]
                df2 = df2.groupby('QTRC_Date End')[['SalesTTM', 'OperatingprofitTTM', 'NetProfit_TTM', 'SalesTTM_Absolute', 'OperatingprofitTTM_Absolute', 'NetProfit_TTM_change_Absolute','Increaseinstock', 'NetProfit_Quarterly_Positives', 'QTRC_Net Sales & Other Operating Income','QTRC_PBIDT (Excl OI)', 'QTRC_Operating Profit', 'QTRC_Consolidated Net Profit']].sum()

                df2['salesTTM%'] = ((df2['SalesTTM'] - df2['SalesTTM'].shift(1)) / df2['SalesTTM'].shift(1)) 
                df2['OPTTM%'] = ((df2['OperatingprofitTTM'] - df2['OperatingprofitTTM'].shift(1)) / df2['OperatingprofitTTM'].shift(1)) 
                df2['NetProfitTTM%'] = ((df2['NetProfit_TTM'] - df2['NetProfit_TTM'].shift(1)) / df2['NetProfit_TTM'].shift(1)) 
                
                df2['Sales_Q'] = ((df2['QTRC_Net Sales & Other Operating Income'] - df2['QTRC_Net Sales & Other Operating Income'].shift(1)) / df2['QTRC_Net Sales & Other Operating Income'].shift(1)) 
                df2['OPwithoutOI_Q'] = ((df2['QTRC_PBIDT (Excl OI)'] - df2['QTRC_PBIDT (Excl OI)'].shift(1)) / df2['QTRC_PBIDT (Excl OI)'].shift(1)) 
                df2['OP_Q'] = ((df2['QTRC_Operating Profit'] - df2['QTRC_Operating Profit'].shift(1)) / df2['QTRC_Operating Profit'].shift(1)) 
                df2['Netprofit_Q'] = ((df2['QTRC_Consolidated Net Profit'] - df2['QTRC_Consolidated Net Profit'].shift(1)) / df2['QTRC_Consolidated Net Profit'].shift(1)) 


                df2 = df2.drop(df2.index[0])
                df2['Date'] = date
                df2['sector'] = sector
                
                df2['CompanyCounts'] = symcounts
                df2['MissingSymb'] = missingsymbs
                df2['MissingOPTTM'] = missingOPTTM
                df2['MissingMCAP'] = missingMcap

                sector_df = pd.concat([sector_df, df2])

            except Exception as e:
                print(f"Error occurred for {date} in sector {sector}: {str(e)}")
                continue

        # Concatenate the results for the current sector to the main results DataFrame
        datesdf = pd.concat([datesdf, sector_df])

    except Exception as e:
        print(f"Error occurred for sector {sector}: {str(e)}")
        continue

datesdf['uniquecode_1'] = datesdf['sector'] + datesdf['Date'].astype(str)
filepath = r'C:\Users\99111\OneDrive\Desktop\daily updates\Quarterly Results\SectorQuarterlyresults.xlsx'
datesdf.to_excel(filepath, index=True)


Error occurred for 2016-03-31T00:00:00.000000000 in sector Healthcare: index 1 is out of bounds for axis 0 with size 1
Error occurred for 2016-03-31T00:00:00.000000000 in sector FMCG: index 1 is out of bounds for axis 0 with size 1
Error occurred for 2016-03-31T00:00:00.000000000 in sector Capital Goods: index 1 is out of bounds for axis 0 with size 1
Error occurred for 2016-03-31T00:00:00.000000000 in sector Infrastructure: index 1 is out of bounds for axis 0 with size 1
Error occurred for 2016-03-31T00:00:00.000000000 in sector Chemicals: index 1 is out of bounds for axis 0 with size 1
Error occurred for 2016-03-31T00:00:00.000000000 in sector Plastic Products: index 1 is out of bounds for axis 0 with size 1
Error occurred for 2016-03-31T00:00:00.000000000 in sector Media & Entertainment: index 1 is out of bounds for axis 0 with size 1
Error occurred for 2016-03-31T00:00:00.000000000 in sector Finance: index 1 is out of bounds for axis 0 with size 1
Error occurred for 2016-03-31T00:0

LOADING DATA FOR COMPANIES WITH IN THE INDUSTRIES

In [5]:
folder_path = "C:/Users/99111/OneDrive/Desktop/Full data set/Price, PE, EBITDA , EV & Mcap/*.csv"

files = glob.glob(folder_path)

df_list = []

for file in files:
    df = pd.read_csv(file)
    df_list.append(df)

df = pd.concat(df_list)

df['NDP_Date'] = pd.to_datetime(df['NDP_Date'])

latest_date = df['NDP_Date'].max()

LatestSheet = df[df['NDP_Date'] == latest_date]
LatestSheet = LatestSheet.loc[:, ['Company Name','CD_NSE Symbol','NDP_Date', 'NDP_Close','CD_Sector', 'CD_Industry', 'NDP_Mcap', 'NDP_Consolidated PE', 'NDP_Consolidated EV EBIDTA']]
LatestSheet = LatestSheet.sort_values(by=['CD_Sector', 'CD_Industry','NDP_Mcap'], ascending=[True, True,False])

save_dir = r'C:\Users\99111\OneDrive\Desktop\daily updates\Quarterly Results'        
chart_file = os.path.join(save_dir, "LATESTCOMPANIES.xlsx")
LatestSheet.to_excel(chart_file, index=False)

######################################


file1 = r'C:\Users\99111\OneDrive\Desktop\daily updates\Quarterly Results\IndustryQuarterlyresults.xlsx'
file2 = r'C:\Users\99111\OneDrive\Desktop\daily updates\Quarterly Results\SectorQuarterlyresults.xlsx'
file3 = r'C:\Users\99111\OneDrive\Desktop\daily updates\Quarterly Results\LATESTCOMPANIES.xlsx'
file4 = r'C:\Users\99111\OneDrive\Desktop\daily updates\Quarterly Results\SortedQuarterlyresults.xlsx'


# Read each Excel file into a DataFrame
df1 = pd.read_excel(file1, sheet_name='Sheet1')
df2 = pd.read_excel(file2, sheet_name='Sheet1')
df3 = pd.read_excel(file3, sheet_name='Sheet1')
df4 = pd.read_excel(file4, sheet_name='Sheet1')

# Specify the full path for the combined file, including the desired folder
combined_file_path = r'C:\Users\99111\OneDrive\Desktop\daily updates\Quarterly Results\combined_file.xlsx'

# Create a Pandas Excel writer using ExcelWriter with the specified file path
with pd.ExcelWriter(combined_file_path, engine='xlsxwriter') as writer:
    # Write each DataFrame to a separate sheet in the same Excel file
    df1.to_excel(writer, sheet_name='IndustryQuarterlyresults', index=False)
    df2.to_excel(writer, sheet_name='SectorQuarterlyresults', index=False)
    df3.to_excel(writer, sheet_name='LATESTCOMPANIES', index=False)
    df4.to_excel(writer, sheet_name='SortedQuarterlyresults', index=False)

# The combined Excel file with three tabs will be saved at the specified path
#print(f"Combined file saved at: {combined_file_path}")

for file_path in [file1, file2, file3, file4]:
    try:
        os.remove(file_path)
        print(f"File deleted: {file_path}")
    except FileNotFoundError:
        print(f"File not found: {file_path}")
    except Exception as e:
        print(f"Error deleting file {file_path}: {e}")


File deleted: C:\Users\99111\OneDrive\Desktop\daily updates\Quarterly Results\IndustryQuarterlyresults.xlsx
File deleted: C:\Users\99111\OneDrive\Desktop\daily updates\Quarterly Results\SectorQuarterlyresults.xlsx
File deleted: C:\Users\99111\OneDrive\Desktop\daily updates\Quarterly Results\LATESTCOMPANIES.xlsx
File deleted: C:\Users\99111\OneDrive\Desktop\daily updates\Quarterly Results\SortedQuarterlyresults.xlsx


FULL MARKET

In [ ]:
#########################################
#FETCHING PROCESSED QUARTERLY RESULTS

file_path1 = r"C:\Users\99111\OneDrive\Desktop\Full data set\Quarterly results processed\Quarterlyresults.xlsx"

rdf = pd.read_excel(file_path1)
symbols = rdf['Company Name'].unique().tolist()
rdf = rdf[['CD_NSE Symbol', 'QTRC_Date End', 'QTRC_Result Date','QTRC_Reserves','SalesTTM','OperatingprofitTTM','NetProfit_TTM','SalesTTM_change','OperatingprofitTTM_change','NetProfit_TTM_change','SalesTTM_Absolute','OperatingprofitTTM_Absolute','NetProfit_TTM_change_Absolute', 'NetProfit_Quarterly_Positives','OperatingGrowth','OperatingGrowth(Excl OI)', 'Increaseinstock', 'Increase stock/Netsales', 'foreignexchangeloans/tExpenditure', 'Miscexpense/tExpenditure', 'otherincomeFlag', 'OffloadingInvst/operatingIncome', 'OffloadingAssets/operatingIncome', 'provisions/OperatingIncome', 'ShadyDeductionsonOperatingIncome', 'ShadyDeductionsonPBT', 'ShadyDeductionsonPAT', 'MinorityInterestFlag','uniquecode','Company Name', 'QTRC_Net Sales & Other Operating Income','QTRC_PBIDT (Excl OI)', 'QTRC_Operating Profit', 'QTRC_Consolidated Net Profit']].copy()

#########################################
#FETCHING LATEST MCAP AND INDUSTRY AND SECTORS

folder_path = "C:/Users/99111/OneDrive/Desktop/Full data set/Price, PE, EBITDA , EV & Mcap/*.csv"

files = glob.glob(folder_path)

df_list = []

for file in files:
    df = pd.read_csv(file)
    df_list.append(df)

Mcap = pd.concat(df_list)

Mcap['NDP_Date'] = pd.to_datetime(Mcap['NDP_Date'])

#Mcap = Mcap.sort_values(by='NDP_Date', ascending=False)
Mcap['uniquecode'] = Mcap['Company Name'] + Mcap['NDP_Date'].astype(str)

Mcap = Mcap.sort_values(by='NDP_Date', ascending=False)
Mcap = Mcap.drop_duplicates(subset='Company Name' , keep= 'first')
Mcap = Mcap[['Company Name', 'NDP_Date','CD_Sector','CD_Industry','NDP_Mcap','NDP_Consolidated PE','NDP_Consolidated EV EBIDTA']].copy()

#########################################
#MERGING THE 2 DATA FRAMES

ff = pd.merge(rdf, Mcap, on='Company Name', how='left')
ff = ff.sort_values(by='QTRC_Date End', ascending=True)
ff['QTRC_Result Date'] = pd.to_datetime(ff['QTRC_Result Date'], errors='coerce')
sectors= ff['CD_Sector'].unique().tolist()
Industries= ff['CD_Industry'].unique().tolist()

#########################################

file_path = r"C:\Users\99111\OneDrive\Desktop\Stocklist.xlsx"
symdf = pd.read_excel(file_path)
symbols = symdf['All'].unique().tolist()

ff = ff[ff['CD_NSE Symbol'].isin(symbols)]

##########################################

        ff1 = ff[ff['QTRC_Date End'] >= pd.to_datetime('2016-01-01')]

        datesdf = pd.DataFrame()

        dates = ff1['QTRC_Date End'].unique()


        for date in dates:
            try:
                dff = ff1[ff1['QTRC_Date End'] == date].copy()
                sym = dff['Company Name'].unique().tolist()

                sff1 = ff1[ff1['Company Name'].isin(sym)]
                sff1 = sff1[sff1['QTRC_Date End'] <= date].copy()

                sff1_p = sff1[sff1['QTRC_Date End'] == sff1['QTRC_Date End'].sort_values(ascending=False).unique()[1]].copy()

                sff1 = sff1[sff1['QTRC_Date End'] == date].copy()

                df = pd.concat([sff1_p, sff1])

                df['Count'] = df['Company Name'].map(df['Company Name'].value_counts())

                ################################
                #Need to add stuff here
                ################################

                df1 = df[df['Count'] == 1].copy()
                missingOPTTM = df1['OperatingprofitTTM']
                missingMcap = df1['NDP_Mcap']
                missingsymbs = df1.shape[0]

                df2 = df[df['Count'] == 2].copy()
                symcounts = df2.shape[0]
                df2 = df2.groupby('QTRC_Date End')[['SalesTTM', 'OperatingprofitTTM', 'NetProfit_TTM', 'SalesTTM_Absolute', 'OperatingprofitTTM_Absolute', 'NetProfit_TTM_change_Absolute','Increaseinstock', 'NetProfit_Quarterly_Positives', 'QTRC_Net Sales & Other Operating Income','QTRC_PBIDT (Excl OI)', 'QTRC_Operating Profit', 'QTRC_Consolidated Net Profit']].sum()

                df2['salesTTM%'] = ((df2['SalesTTM'] - df2['SalesTTM'].shift(1)) / df2['SalesTTM'].shift(1)) 
                df2['OPTTM%'] = ((df2['OperatingprofitTTM'] - df2['OperatingprofitTTM'].shift(1)) / df2['OperatingprofitTTM'].shift(1)) 
                df2['NetProfitTTM%'] = ((df2['NetProfit_TTM'] - df2['NetProfit_TTM'].shift(1)) / df2['NetProfit_TTM'].shift(1)) 
                
                df2['Sales_Q'] = ((df2['QTRC_Net Sales & Other Operating Income'] - df2['QTRC_Net Sales & Other Operating Income'].shift(1)) / df2['QTRC_Net Sales & Other Operating Income'].shift(1)) 
                df2['OPwithoutOI_Q'] = ((df2['QTRC_PBIDT (Excl OI)'] - df2['QTRC_PBIDT (Excl OI)'].shift(1)) / df2['QTRC_PBIDT (Excl OI)'].shift(1)) 
                df2['OP_Q'] = ((df2['QTRC_Operating Profit'] - df2['QTRC_Operating Profit'].shift(1)) / df2['QTRC_Operating Profit'].shift(1)) 
                df2['Netprofit_Q'] = ((df2['QTRC_Consolidated Net Profit'] - df2['QTRC_Consolidated Net Profit'].shift(1)) / df2['QTRC_Consolidated Net Profit'].shift(1)) 


                df2 = df2.drop(df2.index[0])
                df2['Date'] = date
                #df2['sector'] = sector
                
                df2['CompanyCounts'] = symcounts
                df2['MissingSymb'] = missingsymbs
                df2['MissingOPTTM'] = missingOPTTM
                df2['MissingMCAP'] = missingMcap

                datesdf = pd.concat([datesdf, df2])

            except Exception as e:
                print(f"Error occurred for {date} : {str(e)}")
                continue

        datesdf.to_csv('csv.csv')


testing daily details

In [ ]:
#SORTING Quarterly results DF's and Fetching MCAP, PE and EV / EBITDA
########################################
#FETCHING QUARTERLY RESULTS

file_path1 = r"C:\Users\99111\OneDrive\Desktop\Full data set\Quarterly results processed\Quarterlyresults.xlsx"

rdf = pd.read_excel(file_path1)
symbols = rdf['Company Name'].unique().tolist()
rdf = rdf[['CD_NSE Symbol', 'QTRC_Date End', 'QTRC_Result Date','QTRC_Reserves','SalesTTM','OperatingprofitTTM','NetProfit_TTM','SalesTTM_change','OperatingprofitTTM_change','NetProfit_TTM_change','SalesTTM_Absolute','OperatingprofitTTM_Absolute','NetProfit_TTM_change_Absolute', 'NetProfit_Quarterly_Positives','OperatingGrowth','OperatingGrowth(Excl OI)', 'Increaseinstock', 'Increase stock/Netsales', 'foreignexchangeloans/tExpenditure', 'Miscexpense/tExpenditure', 'otherincomeFlag', 'OffloadingInvst/operatingIncome', 'OffloadingAssets/operatingIncome', 'provisions/OperatingIncome', 'ShadyDeductionsonOperatingIncome', 'ShadyDeductionsonPBT', 'ShadyDeductionsonPAT', 'MinorityInterestFlag','uniquecode', 'Company Name']].copy()

#########################################
#FETCHING LATEST MCAP AND INDUSTRY AND SECTORS

folder_path = "C:/Users/99111/OneDrive/Desktop/Full data set/Price, PE, EBITDA , EV & Mcap/*.csv"

files = glob.glob(folder_path)

df_list = []

for file in files:
    df = pd.read_csv(file)
    df_list.append(df)

Mcap = pd.concat(df_list)

Mcap['NDP_Date'] = pd.to_datetime(Mcap['NDP_Date'])

#Mcap = Mcap.sort_values(by='NDP_Date', ascending=False)
Mcap['uniquecode'] = Mcap['Company Name'] + Mcap['NDP_Date'].astype(str)

Mcap = Mcap.sort_values(by='NDP_Date', ascending=False)
Mcap = Mcap.drop_duplicates(subset='Company Name' , keep= 'first')
Mcap = Mcap[['Company Name', 'NDP_Date','CD_Sector','CD_Industry','NDP_Mcap','NDP_Consolidated PE','NDP_Consolidated EV EBIDTA']].copy()

#########################################

ff = pd.merge(rdf, Mcap, on='Company Name', how='left')


In [ ]:
folder_path = "C:/Users/99111/OneDrive/Desktop/Full data set/Price, PE, EBITDA , EV & Mcap/*.csv"

files = glob.glob(folder_path)

df_list = []

for file in files:
    df = pd.read_csv(file)
    df_list.append(df)

Mcap = pd.concat(df_list)

Mcap['NDP_Date'] = pd.to_datetime(Mcap['NDP_Date'])

#Mcap = Mcap.sort_values(by='NDP_Date', ascending=False)
Mcap['uniquecode'] = Mcap['Company Name'] + Mcap['NDP_Date'].astype(str)


In [ ]:
cos = Mcap['Company Name'].unique().tolist()
cos[1]

In [ ]:
mcap1 = Mcap[Mcap['Company Name'] == cos[3]].copy()
mcap1 = mcap1.drop_duplicates(subset='NDP_Date' , keep= 'first')
mcap1 = mcap1.sort_values(by='NDP_Date', ascending=True)

date_range = pd.date_range(start=mcap1['NDP_Date'].min(), end=mcap1['NDP_Date'].max(), freq='D')

# Reindex the DataFrame with the generated date range
mcap1 = mcap1.set_index('NDP_Date').reindex(date_range)

# Forward fill the missing values using the values from the rows above
mcap1 = mcap1.ffill()

# Reset the index to bring 'date' back as a column
mcap1 = mcap1.reset_index()

mcap1.to_csv('csv.csv')

In [ ]:
##########################################################################################

ff1 = Mcap[Mcap['Company Name'] == symbol[1]].copy()

ff1['date'] = pd.to_datetime(ff1['NDP_Date'])

# Sort the DataFrame by date if it's not already sorted
ff1 = ff1.sort_values(by='NDP_Date')

# Generate a date range that spans from the minimum to maximum date in your DataFrame
date_range = pd.date_range(start=ff1['date'].min(), end=ff1['date'].max(), freq='D')

# Reindex the DataFrame with the generated date range
df = df.set_index('date').reindex(date_range)

# Forward fill the missing values using the values from the rows above
df = df.ffill()

# Reset the index to bring 'date' back as a column
df = df.reset_index()

FOR FETCHING FCFE AS WELL

In [ ]:
file_path1 = r"C:\Users\99111\OneDrive\Desktop\Full data set\FCFE\Data\combinedfile.csv"
combinedfile = pd.read_csv(file_path1)

file_path2 = r"C:\Users\99111\OneDrive\Desktop\Full data set\Quarterly results processed\TTMFilter.xlsx"
TTMFilter = pd.read_excel(file_path2)

merged_df = pd.merge(TTMFilter, combinedfile, on='uniquecode', how='outer')
filepath = r'C:\Users\99111\OneDrive\Desktop\Full data set\Quarterly results processed\Quarterly&FCFE.xlsx'
merged_df.to_excel(filepath, index=False)

In [ ]:
final = pd.DataFrame()

file_path = r'C:\Users\99111\OneDrive\Desktop\Full data set\Quarterly results processed\Quarterly&FCFE.xlsx'
df = pd.read_excel(file_path)

symbols = df['Company Name'].unique().tolist()

for symbol in symbols:
    try:
        
        df1 = df[df['Company Name'] == symbol].copy()
        df1 = df1.sort_values('QTRC_Date End')

        df1['Other Income / Operating Income'] = df1['Other Income / Operating Income'].fillna(method='ffill')
        df1['Misc & Capitalised Expense'] = df1['Misc & Capitalised Expense'].fillna(method='ffill')
        df1['change in Misc & Capitalised Expense'] = df1['change in Misc & Capitalised Expense'].fillna(method='ffill')
        df1['Misc & Capitalised Expense / Total Expenditure'] = df1['Misc & Capitalised Expense / Total Expenditure'].fillna(method='ffill')

        df1['Net Sales Growth'] = df1['Net Sales Growth'].fillna(method='ffill')
        df1['Operating Profit Growth'] = df1['Operating Profit Growth'].fillna(method='ffill')
        df1['PAT Growth'] = df1['PAT Growth'].fillna(method='ffill')
        df1['Minority share / PAT'] = df1['Minority share / PAT'].fillna(method='ffill')
        df1['Actual payout to Equity'] = df1['Actual payout to Equity'].fillna(method='ffill')
        df1['Funny Items in operating CFs'] = df1['Funny Items in operating CFs'].fillna(method='ffill')
        df1['change in Funny Items in operating CFs'] = df1['change in Funny Items in operating CFs'].fillna(method='ffill')
        df1['Minority interest  / Total NC Liability'] = df1['Minority interest  / Total NC Liability'].fillna(method='ffill')
        df1['LT Payables & provisions & other'] = df1['LT Payables & provisions & other'].fillna(method='ffill')
        df1['Change in LT Payables & provisions & other'] = df1['Change in LT Payables & provisions & other'].fillna(method='ffill')
        df1['LT Payables & provisions / Total NC liabilities'] = df1['LT Payables & provisions / Total NC liabilities'].fillna(method='ffill')
        df1['Change in Net Block'] = df1['Change in Net Block'].fillna(method='ffill')
        df1['FunnyAssets'] = df1['FunnyAssets'].fillna(method='ffill')
        df1['Change in FunnyAssets'] = df1['Change in FunnyAssets'].fillna(method='ffill')
        df1['FunnyAssets Flag'] = df1['FunnyAssets Flag'].fillna(method='ffill')
        df1['Contingent Liability / Total Assets'] = df1['Contingent Liability / Total Assets'].fillna(method='ffill')
        df1['Contingent liability increase Flag'] = df1['Contingent liability increase Flag'].fillna(method='ffill')
        df1['Non Cash Working Capital'] = df1['Non Cash Working Capital'].fillna(method='ffill')
        df1['Cash/Current Liability'] = df1['Cash/Current Liability'].fillna(method='ffill')

        df1['PLC_Consolidated Net Profit'] = df1['PLC_Consolidated Net Profit'].fillna(method='ffill')
        df1['CAPEX'] = df1['CAPEX'].fillna(method='ffill')
        df1['Net Casflow from Debt'] = df1['Net Casflow from Debt'].fillna(method='ffill')
        df1['NetfromPreferencialEquity'] = df1['NetfromPreferencialEquity'].fillna(method='ffill')
        df1['Change in Non Cash Working Capital'] = df1['Change in Non Cash Working Capital'].fillna(method='ffill')
        df1['FCFE'] = df1['FCFE'].fillna(method='ffill')

        df1['PLC_Net Sales_4y'] = df1['PLC_Net Sales_4y'].fillna(method='ffill')
        df1['PLC_Operating Profit_4y'] = df1['PLC_Operating Profit_4y'].fillna(method='ffill')

        df1['PLC_Consolidated Net Profit_4y'] = df1['PLC_Consolidated Net Profit_4y'].fillna(method='ffill')
        df1['CAPEX_4y'] = df1['CAPEX_4y'].fillna(method='ffill')
        df1['Net Casflow from Debt_4y'] = df1['Net Casflow from Debt_4y'].fillna(method='ffill')
        df1['NetfromPreferencialEquity_4y'] = df1['NetfromPreferencialEquity_4y'].fillna(method='ffill')
        df1['Change in Non Cash Working Capital_4y'] = df1['Change in Non Cash Working Capital_4y'].fillna(method='ffill')
        df1['FCFE_4y'] = df1['FCFE_4y'].fillna(method='ffill')
     
     

        final = pd.concat([final, df1])

    except Exception as e:
        print(f"Error occurred  for {symbol}: {str(e)}")
        continue

final = final[['Company Name','CD_Trading Status','CD_NSE Symbol','QTRC_Date End','QTRC_Result Date','QTRC_Net Sales','QTRC_Operating Profit','Increaseinstock','Increase stock/Netsales','foreignexchangeloans/tExpenditure','Miscexpense/tExpenditure','otherincomeFlag','OffloadingInvst/operatingIncome','OffloadingAssets/operatingIncome','provisions/OperatingIncome','ShadyDeductionsonOperatingIncome','ShadyDeductionsonPBT','ShadyDeductionsonPAT','MinorityInterestFlag','NetProfit_TTM','NetProfit_TTM_change','NetProfit+Flagger','NetProfit-lagger','NetProfit_TTM_Positives','NetProfit_Quarterly_Positives','BSC_Total Assets','BSC_Total Debt','BSC_Book Value','BSC_Adjusted Book Value','Other Income / Operating Income','Misc & Capitalised Expense','change in Misc & Capitalised Expense','Misc & Capitalised Expense / Total Expenditure','Net Sales Growth','Operating Profit Growth','PAT Growth','Minority share / PAT','Actual payout to Equity','Funny Items in operating CFs','change in Funny Items in operating CFs','Minority interest  / Total NC Liability','LT Payables & provisions & other','Change in LT Payables & provisions & other','LT Payables & provisions / Total NC liabilities','Change in Net Block','FunnyAssets','Change in FunnyAssets','FunnyAssets Flag','Contingent Liability / Total Assets','Contingent liability increase Flag','Non Cash Working Capital','Cash/Current Liability','PLC_Consolidated Net Profit','CAPEX','Net Casflow from Debt','NetfromPreferencialEquity','Change in Non Cash Working Capital','PLC_Net Sales_4y','PLC_Operating Profit_4y','PLC_Consolidated Net Profit_4y','CAPEX_4y','Net Casflow from Debt_4y','NetfromPreferencialEquity_4y','Change in Non Cash Working Capital_4y','FCFE_4y','FCFE','uniquecode',]]

filepath = r'C:\Users\99111\OneDrive\Desktop\Full data set\Quarterly results processed\ResultsFile.xlsx'
final.to_excel(filepath, index=False)

